In [1]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [6]:
with open('clean_bb_10k_comments.json') as json_file:  #first 1mill bb comments
    data = json.load(json_file)

In [7]:

bb_tok = Tokenizer(oov_token='<unk>')
bb_tok.fit_on_texts(data)

In [8]:
len(bb_tok.word_index)

10002

In [5]:
%ls 


baby-steps.ipynb              hp-v5.2-tok.pickle
bb-1m-tok.pickle              hp-v5-tok.pickle
bb-500k-tok.pickle            huffpo-data-cleaning.ipynb
bb-50k-tok-base.pickle        huffpo-model-v0.ipynb
bb-50k-tok.pickle             Huffpo Model v1.0.ipynb
bb_all_comments.csv           Huffpo Model v1.0.py
BB Model v1.0.ipynb           huffpo-model-v1.ipynb
BB Model v2.ipynb             huffpo-model-v1.py
BB Model v3.ipynb             Huffpo Model v4.ipynb
BB Model v4.ipynb             Huffpo Model v5.1.ipynb
BB Model v5.ipynb             Huffpo Model v5.2.ipynb
bb-v1-tok.pickle              Huffpo Model v5-Copy1.ipynb
bb-v2-tok.pickle              Huffpo Model v5.ipynb
bb-v3.1-tok.pickle            huffpo-v1-tok.pickle
bb-v5-tok.pickle              huffpo wordvectors v1.ipynb
clean_bb_comments.json        investigation-Copy1.ipynb
clean_hp_1k_comments.json     investigation.ipynb
clean_hp_comments.json        models/
combined-comment-tokens.json  old/
comments_csv                  p

In [9]:
with open('bb-v5.1-tok.pickle', 'wb') as handle:
    pickle.dump(bb_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:


train_data = data

seqs = bb_tok.texts_to_sequences(train_data)
less_20_seqs = list(filter(lambda x: len(x) <20, seqs))

In [11]:
len(less_20_seqs)

654993

In [13]:
MAXLEN = 19
VOCAB = int(len(bb_tok.word_index))
SEED = 42

sequences = pad_sequences(less_20_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [14]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(LSTM(500))



model.add(Dense(VOCAB, activation='softmax'))

print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           3000600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             5011002   
Total params: 9,613,602
Trainable params: 9,613,602
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:

model_json = model.to_json()
with open("models/bb-model-v5.1.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v5.1/best-weights.hdf5" #off of training acc
checkpoint = ModelCheckpoint(fpath, monitor='acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('bb-modelv5.1'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.01, epochs=200, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 551176 samples, validate on 5568 samples
Epoch 1/200
551176/551176 [==============================] - 240s 435us/step - loss: nan - acc: 0.1217 - val_loss: 9.2105 - val_acc: 5.3879e-04

Epoch 00001: acc improved from -inf to 0.12172, saving model to weights/bb-v5.1/best-weights.hdf5
Epoch 2/200
551176/551176 [==============================] - 237s 430us/step - loss: nan - acc: 8.9082e-04 - val_loss: 9.2105 - val_acc: 5.3879e-04

Epoch 00002: acc did not improve from 0.12172
Epoch 3/200
551176/551176 [==============================] - 280s 508us/step - loss: nan - acc: 8.9082e-04 - val_loss: 9.2105 - val_acc: 5.3879e-04

Epoch 00003: acc did not improve from 0.12172
Epoch 4/200
551176/551176 [==============================] - 383s 695us/step - loss: nan - acc: 8.9082e-04 - val_loss: 9.2105 - val_acc: 5.3879e-04

Epoch 00004: acc did not improve from 0.12172
Epoch 5/200
551176/551176 [==============================] - 458s 830us/st